## Data preparation in Excel
<ul>
    <li>In the original file Scotland was duplicated as Scot - changed it to Scotland</li>
    <li>ISO codes were atteched to the countrius using VPR VLOOKUP function</li>
    <li>There was duplicate wrong ISO code for Poland HR, we changed it to PL</li>
    <li>There were some attractions with unspecific names (e.g. old town) we added city names to them</li>
    <li>We removed one place MAG0083 because is not in Europe</li>
    <li>We also have created the file that we open in order to change the name of the location for get it working with geopy</li>
</ul>


In [1]:
import datetime

import pandas as pd
from geopy.geocoders import Nominatim

ModuleNotFoundError: No module named 'pandas'

In [ ]:
ds = pd.read_excel('GeocodingAttractions.xlsx', sheet_name='Attractions', decimal=",", engine='openpyxl')
ds.describe(include='all', datetime_is_numeric=True).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,100,100,MAG001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,100,100,Sagrada Familia,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,100,25,Spain,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ISO,100,25,ES,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
location_lat,99.0,NaN,NaN,NaN,46.394648,6.675416,34.755986,41.106853,45.17588,51.330414,60.397435
location_long,99.0,NaN,NaN,NaN,8.724077,12.367148,-9.396203,-1.701258,6.183286,16.924632,37.62138
location_address,99,98,"Schloss Schönbrunn, 47, Schönbrunner Schlossst...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 7,3,3,105 58,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 8,2,2,Ελλάς,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Adding attraction coordinates
We used geopy and Nominatim library to put coordinates to attractions. <br>
Some of the coordinates were not identified correctly, we checked them manually with the help of OpenStreetMaps, service which is as well based on Nominatim database. After several iterations we were able to match every location with the correct adress. <br>
While working on adresses we found out that attraction MAG083 doesn't belong to Europe, it's located in Curacao, further we excluded it from the dataset
<br>
Next step - we merged the coordinates to "cleanedISO" file with VLOOKUP function


In [ ]:
geolocator = Nominatim(user_agent="coordinates")

for i in ds.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(ds['Name'][i])
        
        #append lat/long to column using dataframe location
        ds.loc[i,'location_lat'] = location.latitude
        ds.loc[i,'location_long'] = location.longitude
        ds.loc[i,'location_address'] = location.address
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        ds.loc[i,'location_lat'] = ""
        ds.loc[i,'location_long'] = ""
        ds.loc[i,'location_address'] = ""

#print first rows as sample
ds.head(100)

,ID,Name,Country,ISO,location_lat,location_long,location_address,Unnamed: 7,Unnamed: 8
0,MAG001,Sagrada Familia,Spain,ES,41.403479,2.17441,"Basílica de la Sagrada Família, 401, Carrer de...",NaN,NaN
1,MAG002,Parc Guell,Spain,ES,41.414235,2.152458,"Park Güell, Carrer d'Olot, la Salut, Gràcia, B...",NaN,NaN
2,MAG003,Tower of London,England,UK,51.50812,-0.076222,"Tower of London, Tower Hill, Tower Liberty, Wh...",NaN,NaN
3,MAG004,Casa Batllo,Spain,ES,41.391545,2.164696,"Casa Batlló, 43, Passeig de Gràcia, la Dreta d...",NaN,NaN
4,MAG005,Staromestske namesti,Czech Republic,CZ,50.087455,14.420967,"Staroměstské náměstí, Staré Město, Hlavní měst...",NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,MAG096,Duomo di Pisa,Italy,IT,43.723283,10.395728,"Cattedrale di Santa Maria Assunta, Piazza del ...",NaN,NaN
96,MAG097,La Lonja de la Seda,Spain,ES,39.474237,-0.378562,"La Llotja de la Seda, Plaça del Mercat, el Mer...",NaN,NaN
97,MAG098,Ávila Walls,Spain,ES,40.656417,-4.707537,"Murallas de Ávila, Bajada San Nicolás, Ávila, ...",NaN,NaN
98,MAG099,Place Stanislas,France,FR,48.693524,6.183286,"Place Stanislas, Saint-Nicolas - Charles III -...",NaN,NaN


In [ ]:
# dsNoOutlier.to_excel('cleaned.xlsx', index=False, sheet_name='Reviews')
ds.to_excel('GeocodingAttractions.xlsx', index=False, sheet_name='Attractions')

print(datetime.datetime.now())

2022-12-07 13:14:29.160195
